In [ ]:
import pandas as pd

def read_vcf_to_excel(path, out_path):
    import io
    import os
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
        
    vcf_df = pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t').rename(columns={'#CHROM': 'CHROM'})
    
    vcf_df = vcf_df.assign(ANNOTATION=vcf_df['INFO'].str.extract(r'ANN=(.*)'))
    # shift column 'Name' to first position
    first_column = vcf_df.pop('ANNOTATION')
    # insert column using insert(position,column_name,
    # first_column) function
    vcf_df.insert(8, 'ANNOTATION', first_column)
    vcf_df.to_excel(out_path)
    
    return vcf_df

In [ ]:
dataset = "gaard-sanger"
wkdir = ""

# SNP dataframe

In this notebook, we display the variant calling and annotation results as a pandas dataframe and save it as an excel spreadsheet for the user to explore or analyse further. If the DataFrame is too large to display, please use the .xlsx file. 

#### Target variant calling

In [ ]:
vcf_df = read_vcf_to_excel(
    path=f"results/vcfs/targets/{dataset}.annot.vcf",
    out_path=f"results/vcfs/targets/{dataset}-snps.xlsx"
)
pd.set_option("display.max_rows", 1000, "display.max_columns", 1000)
vcf_df

---

In [ ]:
from IPython.display import display, Markdown
display(Markdown(f'<a href="{wkdir}/results/vcfs/targets/{dataset}-snps.xlsx">Target SNP data(.xlsx)</a>'))

#### Whole-amplicon variant calling

In [ ]:
vcf_df = read_vcf_to_excel(
    path=f"results/vcfs/amplicons/{dataset}.annot.vcf",
    out_path=f"results/vcfs/amplicons/{dataset}-snps.xlsx"
)
vcf_df

---

In [ ]:
display(Markdown(f'<a href="{wkdir}/results/vcfs/amplicons/{dataset}-snps.xlsx">Whole amplicon SNP data(.xlsx)</a>'))